# Testing Evidential Classification — Step-by-Step Tutorial

This notebook explains and tests two core components of the `probly` library:

1. The registration system in `common.py`  
2. The Torch appender in `torch.py`  

We'll:
- Understand how the registration works  
- Verify that the Torch appender adds a Softplus activation  
- Create and run custom tests with real data  

At the end, you'll know how to register your own appender and test the entire pipeline end-to-end.


In [14]:
import torch
from torch import nn

from lazy_dispatch.isinstance import LazyType
from probly.transformation.evidential.classification.common import (
    evidential_classification,
    register,
)
from probly.transformation.evidential.classification.torch import append_activation_torch

print("Imports successful.")

Imports successful.


## Step 1 — Understanding `register()` and `evidential_classification()`

In the `common.py` file, Probly defines a small internal registry system.

| Function | Purpose | Analogy |
|-----------|----------|---------|
| `register(cls, appender)` | Stores a mapping between a model class and its appender function | Like a contact list — “If you see this model type, call this function.” |
| `evidential_classification(model)` | Looks up which appender is registered for the model’s type and applies it | Like a dispatcher that finds the correct plugin automatically |

Let's test that this logic works by creating a dummy model and a fake appender.


In [15]:
class DummyModel:
    """A simple dummy model for testing."""


def fake_appender(model: LazyType) -> str:
    print("Appender called for", type(model).__name__)
    return f"Appended({type(model).__name__})"


# Register the dummy model
register(DummyModel, fake_appender)

# Run the dispatcher
result = evidential_classification(DummyModel())
print("Result:", result)

Appender called for DummyModel
Result: Appended(DummyModel)


The registration works as expected.

Here’s what happened:
- You registered the `DummyModel` with a simple appender function.
- When calling `evidential_classification(DummyModel())`, it automatically found the correct appender and executed it.

Now let's test what happens if we call it with an unregistered type — this should raise a clear error.


In [6]:
class UnknownModel:
    pass


try:
    evidential_classification(UnknownModel())
except NotImplementedError as e:
    print("Expected error:", e)

Expected error: No evidential classification appender registered for type <class '__main__.UnknownModel'>


## Step 2 — Testing `append_activation_torch()`

In `torch.py`, Probly defines a simple helper function:

```python
def append_activation_torch(obj: nn.Module) -> nn.Sequential:
    return nn.Sequential(obj, nn.Softplus())


In [19]:
base = nn.Linear(4, 2)
model = append_activation_torch(base)

print(model)

assert isinstance(model, nn.Sequential)  # noqa: S101
assert isinstance(model[0], nn.Linear)  # noqa: S101
assert isinstance(model[1], nn.Softplus)  # noqa: S101

print("Torch appender structure looks correct.")

Sequential(
  (0): Linear(in_features=4, out_features=2, bias=True)
  (1): Softplus(beta=1.0, threshold=20.0)
)
Torch appender structure looks correct.


The test passed successfully.

You just verified that:
- The appender correctly returns a `torch.nn.Sequential`
- The base model remains the first layer
- A `Softplus` is added as the final activation

Now let's confirm that the `evidential_classification()` function automatically uses this appender for any PyTorch model.


In [20]:
torch_model = nn.Linear(10, 3)
wrapped = evidential_classification(torch_model)

print(wrapped)
assert isinstance(wrapped, nn.Sequential)  # noqa: S101
assert isinstance(wrapped[-1], nn.Softplus)  # noqa: S101
print("evidential_classification() correctly used the Torch appender.")

Sequential(
  (0): Linear(in_features=10, out_features=3, bias=True)
  (1): Softplus(beta=1.0, threshold=20.0)
)
evidential_classification() correctly used the Torch appender.


## Step 3 — Custom Test with Real Data

Let's now create a small fake dataset and feed it through our evidential model.

This helps verify that:
- The output works numerically
- The `Softplus` activation ensures all outputs are non-negative


In [ ]:
# Simple test model
net = nn.Sequential(nn.Linear(2, 2), nn.ReLU())

# Make it evidential
ev_model = evidential_classification(net)

# Create dummy input data
x = torch.tensor([[0.5, 1.0], [2.0, -1.0], [1.5, 3.0]])

# Run forward pass
with torch.no_grad():
    y = ev_model(x)

print("Input:\n", x)
print("Output:\n", y)

# Check that all outputs are positive
assert torch.all(y >= 0)  # noqa: S101
print("All outputs are non-negative. Softplus confirmed.")

Input:
 tensor([[ 0.5000,  1.0000],
        [ 2.0000, -1.0000],
        [ 1.5000,  3.0000]])
Output:
 tensor([[0.6931, 0.6931],
        [0.6931, 0.6931],
        [0.6931, 0.6931]])
All outputs are non-negative. Softplus confirmed.


## Step 4 — Recap

| File | Functionality Tested | Expected Behavior |
|------|----------------------|-------------------|
| `common.py` | Appender registration system | Maps model classes to the correct appender |
| `torch.py` | Torch appender function | Adds a Softplus activation |
| Integration test | Combined behavior | Torch models automatically use the correct appender |
| Data test | Forward pass | All outputs are positive due to Softplus |

Everything works as expected.

You now know:
- How the internal registry in `common.py` works
- How the Torch appender adds the activation
- How to test it interactively with Jupyter and real data

In [ ]:
# Define your own small model
class MyNet(nn.Module):
    def __init__(self) -> None:  # noqa: D107
        super().__init__()
        self.fc1 = nn.Linear(3, 4)
        self.fc2 = nn.Linear(4, 2)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward pass.

        Args:
            x: torch.Tensor, input data
        Returns:
            torch.Tensor, output data
        """
        # //TODO(@todo): fill in your own 2-line forward pass below
        # HINT: use ReLU and pass through both layers
        x = torch.relu(self.fc1(x))  # <- first line
        x = self.fc2(x)  # <- second line
        return x


# Create model and make it evidential
my_net = MyNet()
ev_model = evidential_classification(my_net)

# Create dummy input data
x = torch.randn(4, 3)

with torch.no_grad():
    y = ev_model(x)

print("Input:\n", x)
print("Output:\n", y)

# Check that all outputs are positive
assert torch.all(y >= 0)  # noqa: S101
print("All outputs are non-negative. Softplus confirmed.")

Input:
 tensor([[ 0.4787,  0.4512,  0.2799],
        [ 0.0497, -1.6902,  0.2462],
        [-0.8920, -1.7543, -0.9900],
        [ 0.4936,  0.2649, -0.2056]])
Output:
 tensor([[0.8671, 1.0110],
        [0.7782, 0.8960],
        [0.8912, 0.9888],
        [0.8958, 1.0935]])
All outputs are non-negative. Softplus confirmed.
